# The Miller-Rabin algorithm
Large prime numbers are used in almost every cryptosystem today. One problem that is very prominent in the field of security is the challenge of verifying whether a given number is prime or not. This problem is very challenging due to the sheer scale of the prime numbers, for example the RSA algorithm uses a key of length 1024 bits (minimum), and so we need two primes of approximately half that length. In powers of 10, this is 150 digits. To check if a number is prime, we need to check if the given number is divisible by all numbers less than or equal to the square root of the given number. The square root of a 150 bit number is a 75 bit number, and exploring all the possibilities would take around 10000 years on a classical machine!  
To overcome this hurdle and test for primality in acceptable time, we have the Miller-Rabin primality test that never declares a prime number as composite, but may declare a composite number as prime. This method of primality testing results in a correct guess 50% of the time, which means that if we run the test n times, our probability of being correct is 1 - 1/2<sup>n</sup>.  
A deterministic approach to determine whether a number is prime or not exists (AKS test) but it takes poly(m) time where m is the number of bits in the number. For most use cases this is much too slow and we opt for the probabalistic method.

## How does it work?
The Miller-Rabin test relies on a set of equalities that hold for prime numbers, but may not hold for composite numbers. 

- For a prime number p, a<sup>p-1</sup> $\cong$ 1 mod p for all a in {1, 2, ... p-1}
- For a composite number N (except Carmichael numbers) atleast half the elements a in {1, 2, 3 ... N-1} have a<sup>N-1</sup> $\neq$ 1 mod N
- So if we randomly pick a number in {1, 2, ... N-1} and compute a<sup>N-1</sup> mod N : 
    * If N is a prime number, then it is always 1
    * If N is a composite number (excluding carmichael numbers) then it is not 1 with probability 1/2.

The miller rabin test uses the above to test if a number is prime or not. However the algorithm fails for carmichael numbers (they emulate the property of prime numbers) but carmichael numbers are very rare but infinitely many. A seperate test may have to be done for carmichael numbers. 

In [17]:
import random

In [18]:
# First we need a power function which we will create using a binary exponentiation method
def power(x, y, mod):
    res = 1
    x = x%mod
    
    while y > 0:
        if y & 1:
            res *= x
            res %= mod
        
        y = y>>1
        x *= x
        x %= mod
    
    return res

# Here d is the largest odd number that divides n-1.
# If the function returns true, our number is prime otherwise it is composite
def MillerRabin(n, d):
    
    a = 2 + random.randint(1, n-4)
    
    x = power(a, d, n)
    
    if x == 1 or x == n-1:
        return True
    
    while d != n-1:
        x = (x*x)%n
        d *= 2
        
        if x == 1:
            return False
        if x == n-1:
            return True
        
    return False

# n is the number, k is the number of times we run the test
def CheckPrimality(n, k):
    
    if n <= 1 or n == 4:
        return False
    if n <= 3:
        return True
    
    d = n-1
    while d%2 == 0:
        d //= 2
    
    for i in range(0, k):
        if MillerRabin(n, d) == False:
            return False
    
    return True

Now let us test this code on numbers < 1000

In [20]:
Primes = []

for n in range(1, 10000):
    if CheckPrimality(n, 10):
        Primes.append(n)
print(Primes[0:10])

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29]


As we can see, the algorithm returns the first 10 primes correctly. However let us test on a large prime number, a large composite number and a carmichael number.

In [24]:
# Large prime number
if CheckPrimality(100000004861, 10):
    print("100000004861 is prime")
else:
    print("100000004861 is not prime")

# Large composite number (odd)
if CheckPrimality(3628809, 10):
    print("3628809 is prime")
else:
    print("3628809 is not prime")

# Carmichael number
if CheckPrimality(75361, 10):
    print("75361 is prime")
else:
    print("75361 is not prime")

100000004861 is prime
3628809 is not prime
75361 is not prime


As we can see, the algorithm gives us the correct output. The probability of it giving us an incorrect output is 1/2<sup>10</sup> which is extremely low, and can be made lower by increasing the value of k.